## Install PGMPY library  https://github.com/pgmpy/pgmpy
### Documentation for this library is at http://pgmpy.org/

In [1]:
!pip install pgmpy

In [2]:
# Import relevant packages
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
import sys

## Create the Bayesian network
### Check example usage at: https://github.com/pgmpy/pgmpy_notebook/blob/master/notebooks/2.%20Bayesian%20Networks.ipynb

In [3]:
# We first create a model which containts edges of the graph
model = BayesianModel([('S','Y'),
                       ('S','W'),
                       ('S','C'),
                       ('F','R'),
                       ('M','R'),
                       ('R','C')])

# Enter conditional probability distribution for each variable

# Prior probability for smoking P(S)
cpd_S = TabularCPD(variable='S', variable_card=2, values=[[0.85],[0.15]])

# Prior probability for solar flare P(F)
cpd_F = TabularCPD(variable='F', variable_card=2, values=[[0.99],[0.01]])

# Prior probability for solar flare P(M)
cpd_M = TabularCPD(variable='M', variable_card=2, values=[[0.05],[0.95]])

### Enter conditional probability in the format shown below. In pgmpy, the colums are the values of parent variables and rows are the values of the  variable whose CPD you're are writing.

$\text{P(Alarm | Burglary, Quake)} = $
\begin{array}{|c|c|c|c|c|}
\hline
\text{Burg} & \text{Burg_0} & \text{Burg_0} & \text{Burg_1} & \text{Burg_1} \\
\hline
\text{Quake} & \text{Quake_0} & \text{Quake_1} & \text{Quake_0} & \text{Quake_1} \\
\hline
\text{Alarm_0} & 0.999 & 0.71 & 0.06  & 0.05 \\
\hline
\text{Alarm_1} & 0.001 & 0.29 & 0.94 & 0.95 \\
\hline
\end{array}

In [4]:
# Conditional probability for W or P(W|S)

cpd_W = TabularCPD(variable='W',variable_card=2,
                   values = [[0.8,0.1],
                             [0.2,0.9]],
                   evidence = ['S'],
                   evidence_card=[2])

# Conditional probability for R or P(R|F,M)

cpd_R = TabularCPD(variable='R', variable_card=2,
                   values = [[0.9, 0.8, 0.8, 0.1],
                             [0.1, 0.2, 0.2, 0.9]],
                   evidence = ['F','M'],
                   evidence_card=[2,2])

# Conditional probability for C or P(C|S,R)

cpd_C = TabularCPD(variable='C', variable_card=2, 
                   values = [[0.9, 0.4, 0.7, 0.1],
                             [0.1, 0.6, 0.3, 0.9]],                             
                   evidence = ['S','R'],
                   evidence_card=[2,2])

# Conditional probability for Y or P(Y|S)
cpd_Y = TabularCPD(variable='Y',variable_card=2,
                   values = [[0.89,0.2],
                             [0.11,0.8]],
                   evidence = ['S'], 
                   evidence_card=[2])

In [5]:
model.add_cpds(cpd_S, cpd_F, cpd_M, cpd_W, cpd_R, cpd_C, cpd_Y)

## Validate network parameters

In [6]:
print(model.check_model())

# cpd = model.get_cpds('Alarm')
# print('vars:', cpd.variable)

True


In [7]:
cpds = model.get_cpds()
for cpd in cpds:
    evidence = ",".join(cpd.variables[1:])
    if evidence:
        print(f"P({cpd.variables[0]}|{evidence})")
    else:
        print(f"P({cpd.variables[0]})")
    print(cpd)

P(S)
+------+------+
| S(0) | 0.85 |
+------+------+
| S(1) | 0.15 |
+------+------+
P(F)
+------+------+
| F(0) | 0.99 |
+------+------+
| F(1) | 0.01 |
+------+------+
P(M)
+------+------+
| M(0) | 0.05 |
+------+------+
| M(1) | 0.95 |
+------+------+
P(W|S)
+------+------+------+
| S    | S(0) | S(1) |
+------+------+------+
| W(0) | 0.8  | 0.1  |
+------+------+------+
| W(1) | 0.2  | 0.9  |
+------+------+------+
P(R|F,M)
+------+------+------+------+------+
| F    | F(0) | F(0) | F(1) | F(1) |
+------+------+------+------+------+
| M    | M(0) | M(1) | M(0) | M(1) |
+------+------+------+------+------+
| R(0) | 0.9  | 0.8  | 0.8  | 0.1  |
+------+------+------+------+------+
| R(1) | 0.1  | 0.2  | 0.2  | 0.9  |
+------+------+------+------+------+
P(C|S,R)
+------+------+------+------+------+
| S    | S(0) | S(0) | S(1) | S(1) |
+------+------+------+------+------+
| R    | R(0) | R(1) | R(0) | R(1) |
+------+------+------+------+------+
| C(0) | 0.9  | 0.4  | 0.7  | 0.1  |
+---

In [8]:
from pgmpy.inference import VariableElimination

# Going to do variable elimination
infer = VariableElimination(model)

In [9]:
phi_query = infer.query(['C'], evidence={'W':1}, joint = False)
factor = phi_query['C']
print('Probability C|W')
print(factor)

Eliminating: S: 100%|███████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 384.62it/s]

Probability C|W
+------+----------+
| C    |   phi(C) |
+======+==========+
| C(0) |   0.7017 |
+------+----------+
| C(1) |   0.2983 |
+------+----------+


In [10]:
phi_query = infer.query(['S'], evidence={'C':1}, joint = False)
factor = phi_query['S']
print('Probability S|C')
print(factor)


Finding Elimination Order: :   0%|                                                               | 0/5 [00:00<?, ?it/s]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Eliminating: W:   0%|                                                                            | 0/5 [00:00<?, ?it/s]

Eliminating: Y:   0%|                                                                            | 0/5 [00:00<?, ?it/s]

Eliminating: M:   0%|                                                                            | 0/5 [00:00<?, ?it/s]

Eliminating: F:   0%|                                                                            | 0/5 [00:00<?, ?it/s]

Finding Elimination Order: : 100%|██████████████████████████████████████████████████████| 5/5 [00:00<00:00, 250.00it/s]

Probability S|C
+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.7300 |
+------+----------+
| S(1) |   0.2700 |
+------+----------+


In [15]:
phi_query = infer.query(['S','R'], evidence={'C':1}, joint = False)
factor_S = phi_query['S']
factor_R = phi_query['R']

print(factor_S)
print(factor_R)

Eliminating: Y: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 444.24it/s]

+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.7300 |
+------+----------+
| S(1) |   0.2700 |
+------+----------+
+------+----------+
| R    |   phi(R) |
+======+==========+
| R(0) |   0.4437 |
+------+----------+
| R(1) |   0.5563 |
+------+----------+


In [12]:
phi_query

{'S': <DiscreteFactor representing phi(S:2) at 0x244c38538e0>,
 'R': <DiscreteFactor representing phi(R:2) at 0x244c3853970>}

In [23]:
phi_query = infer.query(['C'],evidence={"M":0})
print(phi_query)


Finding Elimination Order: :   0%|                                                               | 0/5 [00:00<?, ?it/s]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Eliminating: W:   0%|                                                                            | 0/5 [00:00<?, ?it/s]

Eliminating: Y:   0%|                                                                            | 0/5 [00:00<?, ?it/s]

Eliminating: S:   0%|                                                                            | 0/5 [00:00<?, ?it/s]

Eliminating: F:   0%|                                                                            | 0/5 [00:00<?, ?it/s]

Eliminating: R: 100%|███████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 499.86it/s]

+------+----------+
| C    |   phi(C) |
+======+==========+
| C(0) |   0.8180 |
+------+----------+
| C(1) |   0.1820 |
+------+----------+
